### Association rules by Top 5 popular Departments 
#### Department - Beverages

In [1]:
##importing libraries
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import OnehotTransactions
import pandas as pd
from sklearn import cluster, datasets
import numpy as np
import scipy
from scipy.cluster.vq import kmeans,vq
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

In [3]:
# Csv contains data of trainorders and prior orders
combine_train_prior=pd.read_csv("CombineOrders1.csv",encoding = "ISO-8859-1")

In [4]:
## altering datatypes for ease of use
combine_train_prior['aisle_id']=combine_train_prior['aisle_id'].astype('int16')
combine_train_prior['order_id']=combine_train_prior['order_id'].astype('int32')
combine_train_prior['product_id']=combine_train_prior['product_id'].astype('int32')
combine_train_prior['add_to_cart_order']=combine_train_prior['add_to_cart_order'].astype('int16')
combine_train_prior['reordered']=combine_train_prior['reordered'].astype('int8')
combine_train_prior['department_id']=combine_train_prior['department_id'].astype('int8')



In [5]:
combine_train_prior['product_name'] = combine_train_prior['product_name'].str.strip()
combine_train_prior.dropna(axis=0, subset=['order_id'], inplace=True)

In [6]:
## this will give us the popular departments as departments with the highest value counts will be those from where consumers have purchased the most
##Top 5 Departments are produce,dairy eggs, snacks, beverages, frozen
combine_train_prior['department'].value_counts()

produce            9888378
dairy eggs         5631067
snacks             3006412
beverages          2804175
frozen             2336858
pantry             1956819
bakery             1225181
canned goods       1114857
deli               1095540
dry goods pasta     905340
household           774652
meat seafood        739238
breakfast           739069
personal care       468693
babies              438743
international       281155
alcohol             159294
pets                102221
missing              77396
other                38086
bulk                 35932
Name: department, dtype: int64

In [ ]:
combine_train_prior

In [7]:
## Using Apriori we will find if there are association rules found for products bought from department Dairy Eggs
dept_produce=combine_train_prior[combine_train_prior['department'] =="beverages"].reset_index()

In [8]:
## Preparing dataset for Apriori
temp_df=dept_produce.loc[:,['order_id','product_name']]

In [9]:
## In order to find association we have considered those orders that have products which 4 or more in number
grouped_data=temp_df.groupby("order_id").filter(lambda x: len(x) >=4)

In [10]:
grouped_data

,order_id,product_name
1,98,Organic Raw Kombucha Gingerade
2,98,Organic Orange Juice With Calcium & Vitamin D
3,98,Organic Lemonade
4,98,Natural Spring Water
11,915,Diet Cranberry Fruit Juice
12,915,Soda
13,915,Lemon-Lime Soda
14,915,Diet Cola
15,915,Pure Leaf Unsweetened Iced Tea
16,1032,Amaaazing Blueberry Apple Juice Blend Drink


In [11]:
## Pivoting the table to get occurences of a product in a particuler order
from pandas import *
# table=pd.DataFrame()
table1=pivot_table(grouped_data,index='order_id',columns='product_name',aggfunc=len)

In [13]:
#relace NAN with 0
table1=table1.fillna(0)

In [14]:
## of the product exists then value changed to 1 else 0
def encode_units(x):
    if x == 0.0:
        return 0
    if x >= 1.0:
        return 1


In [15]:
basket_sets = table1.applymap(encode_units)

In [ ]:
basket_sets

In [16]:
## Running Apriori to get frequent itemset with a minimum suppirt of 0.01. ie the combination must exist atleast in 1% of dataset
frequent_itemsets = apriori(table1, min_support=0.01, use_colnames=True)

In [17]:
frequent_itemsets

,support,itemsets
0,0.048241,[100% Raw Coconut Water]
1,0.014753,[All Natural Apricot Sparkling Water]
2,0.010263,[Alpine Spring Water]
3,0.013230,[Apple Juice]
4,0.016917,[Arancita Rossa]
5,0.012244,[Beet Apple Carrot Lemon Ginger Organic Cold P...
6,0.034905,[Blackberry Cucumber Sparkling Water]
7,0.015098,[Canned Aranciata Orange]
8,0.013463,[Classic Soda]
9,0.012018,[Coconut Sparkling Water]


In [20]:
# Getting the rules of association based on the minimum support selected
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [21]:
rules

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(Sparkling Lemon Water, Pure Sparkling Water, ...",(Sparkling Water Grapefruit),0.012448,0.134573,0.010023,0.805210,5.983442,0.008348,4.442861
1,"(Sparkling Lemon Water, Pure Sparkling Water, ...",(Lime Sparkling Water),0.015422,0.098216,0.010023,0.649909,6.617137,0.008508,2.575853
2,"(Sparkling Lemon Water, Lime Sparkling Water, ...",(Pure Sparkling Water),0.027899,0.052498,0.010023,0.359272,6.843492,0.008559,1.478790
3,"(Pure Sparkling Water, Lime Sparkling Water, S...",(Sparkling Lemon Water),0.015401,0.083012,0.010023,0.650801,7.839838,0.008745,2.625975
4,"(Sparkling Lemon Water, Pure Sparkling Water)","(Lime Sparkling Water, Sparkling Water Grapefr...",0.021421,0.055001,0.010023,0.467917,8.507480,0.008845,1.776038
5,"(Sparkling Lemon Water, Lime Sparkling Water)","(Pure Sparkling Water, Sparkling Water Grapefr...",0.040981,0.029040,0.010023,0.244582,8.422111,0.008833,1.285328
6,"(Sparkling Lemon Water, Sparkling Water Grapef...","(Pure Sparkling Water, Lime Sparkling Water)",0.045126,0.021026,0.010023,0.222118,10.563901,0.009074,1.258512
7,"(Pure Sparkling Water, Lime Sparkling Water)","(Sparkling Lemon Water, Sparkling Water Grapef...",0.021026,0.045126,0.010023,0.476701,10.563901,0.009074,1.824722
8,"(Pure Sparkling Water, Sparkling Water Grapefr...","(Sparkling Lemon Water, Lime Sparkling Water)",0.029040,0.040981,0.010023,0.345146,8.422111,0.008833,1.464477
9,"(Lime Sparkling Water, Sparkling Water Grapefr...","(Sparkling Lemon Water, Pure Sparkling Water)",0.055001,0.021421,0.010023,0.182238,8.507480,0.008845,1.196655


In [22]:
## Since, ideally Lift must greater than 1 , we have selected the rules to have Lift to be greater than 1, confidence greater than 30% and a conviction of 40%
final_result=rules[ (rules['lift'] > 1) &
       (rules['confidence'] >= 0.3) & rules['conviction']<1.4 ]
final_result

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(Sparkling Lemon Water, Pure Sparkling Water, ...",(Sparkling Water Grapefruit),0.012448,0.134573,0.010023,0.805210,5.983442,0.008348,4.442861
1,"(Sparkling Lemon Water, Pure Sparkling Water, ...",(Lime Sparkling Water),0.015422,0.098216,0.010023,0.649909,6.617137,0.008508,2.575853
2,"(Sparkling Lemon Water, Lime Sparkling Water, ...",(Pure Sparkling Water),0.027899,0.052498,0.010023,0.359272,6.843492,0.008559,1.478790
3,"(Pure Sparkling Water, Lime Sparkling Water, S...",(Sparkling Lemon Water),0.015401,0.083012,0.010023,0.650801,7.839838,0.008745,2.625975
4,"(Sparkling Lemon Water, Pure Sparkling Water)","(Lime Sparkling Water, Sparkling Water Grapefr...",0.021421,0.055001,0.010023,0.467917,8.507480,0.008845,1.776038
5,"(Sparkling Lemon Water, Lime Sparkling Water)","(Pure Sparkling Water, Sparkling Water Grapefr...",0.040981,0.029040,0.010023,0.244582,8.422111,0.008833,1.285328
6,"(Sparkling Lemon Water, Sparkling Water Grapef...","(Pure Sparkling Water, Lime Sparkling Water)",0.045126,0.021026,0.010023,0.222118,10.563901,0.009074,1.258512
7,"(Pure Sparkling Water, Lime Sparkling Water)","(Sparkling Lemon Water, Sparkling Water Grapef...",0.021026,0.045126,0.010023,0.476701,10.563901,0.009074,1.824722
8,"(Pure Sparkling Water, Sparkling Water Grapefr...","(Sparkling Lemon Water, Lime Sparkling Water)",0.029040,0.040981,0.010023,0.345146,8.422111,0.008833,1.464477
9,"(Lime Sparkling Water, Sparkling Water Grapefr...","(Sparkling Lemon Water, Pure Sparkling Water)",0.055001,0.021421,0.010023,0.182238,8.507480,0.008845,1.196655


In [24]:
final_result.to_csv("Department_BeveragesApriori.csv",sep=',')

In [25]:
test1=pd.read_csv("Department_BeveragesApriori.csv")

In [26]:
## Cleaning the resultset for usability
test1=test1.replace({'frozenset': ''}, regex=True)

In [27]:
test1=test1.replace({"u'": ''}, regex=True)

In [28]:
test1['antecedants'] = test1['antecedants'].str.strip('()').str.strip('[]')
test1['antecedants'] = test1['antecedants'].str.strip('[]')
test1['consequents'] = test1['consequents'].str.strip("'")
test1['antecedants'] = test1['antecedants'].str.replace("'",'')

In [29]:
test1['consequents'] = test1['consequents'].str.strip('()')
test1['consequents'] = test1['consequents'].str.strip('[]')
test1['consequents'] = test1['consequents'].str.strip("'")
test1['consequents'] = test1['consequents'].str.replace("'",'')

In [30]:
test1=test1.drop(['Unnamed: 0'], axis = 1)

In [31]:
test1

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"Sparkling Lemon Water, Pure Sparkling Water, L...",Sparkling Water Grapefruit,0.012448,0.134573,0.010023,0.805210,5.983442,0.008348,4.442861
1,"Sparkling Lemon Water, Pure Sparkling Water, S...",Lime Sparkling Water,0.015422,0.098216,0.010023,0.649909,6.617137,0.008508,2.575853
2,"Sparkling Lemon Water, Lime Sparkling Water, S...",Pure Sparkling Water,0.027899,0.052498,0.010023,0.359272,6.843492,0.008559,1.478790
3,"Pure Sparkling Water, Lime Sparkling Water, Sp...",Sparkling Lemon Water,0.015401,0.083012,0.010023,0.650801,7.839838,0.008745,2.625975
4,"Sparkling Lemon Water, Pure Sparkling Water","Lime Sparkling Water, Sparkling Water Grapefruit",0.021421,0.055001,0.010023,0.467917,8.507480,0.008845,1.776038
5,"Sparkling Lemon Water, Lime Sparkling Water","Pure Sparkling Water, Sparkling Water Grapefruit",0.040981,0.029040,0.010023,0.244582,8.422111,0.008833,1.285328
6,"Sparkling Lemon Water, Sparkling Water Grapefruit","Pure Sparkling Water, Lime Sparkling Water",0.045126,0.021026,0.010023,0.222118,10.563901,0.009074,1.258512
7,"Pure Sparkling Water, Lime Sparkling Water","Sparkling Lemon Water, Sparkling Water Grapefruit",0.021026,0.045126,0.010023,0.476701,10.563901,0.009074,1.824722
8,"Pure Sparkling Water, Sparkling Water Grapefruit","Sparkling Lemon Water, Lime Sparkling Water",0.029040,0.040981,0.010023,0.345146,8.422111,0.008833,1.464477
9,"Lime Sparkling Water, Sparkling Water Grapefruit","Sparkling Lemon Water, Pure Sparkling Water",0.055001,0.021421,0.010023,0.182238,8.507480,0.008845,1.196655


In [32]:
## Writing results to a csv file
test1.to_csv("FinalDepartment_BeveragesApriori.csv")

In [34]:
test1

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"Sparkling Lemon Water, Pure Sparkling Water, L...",Sparkling Water Grapefruit,0.012448,0.134573,0.010023,0.805210,5.983442,0.008348,4.442861
1,"Sparkling Lemon Water, Pure Sparkling Water, S...",Lime Sparkling Water,0.015422,0.098216,0.010023,0.649909,6.617137,0.008508,2.575853
2,"Sparkling Lemon Water, Lime Sparkling Water, S...",Pure Sparkling Water,0.027899,0.052498,0.010023,0.359272,6.843492,0.008559,1.478790
3,"Pure Sparkling Water, Lime Sparkling Water, Sp...",Sparkling Lemon Water,0.015401,0.083012,0.010023,0.650801,7.839838,0.008745,2.625975
4,"Sparkling Lemon Water, Pure Sparkling Water","Lime Sparkling Water, Sparkling Water Grapefruit",0.021421,0.055001,0.010023,0.467917,8.507480,0.008845,1.776038
5,"Sparkling Lemon Water, Lime Sparkling Water","Pure Sparkling Water, Sparkling Water Grapefruit",0.040981,0.029040,0.010023,0.244582,8.422111,0.008833,1.285328
6,"Sparkling Lemon Water, Sparkling Water Grapefruit","Pure Sparkling Water, Lime Sparkling Water",0.045126,0.021026,0.010023,0.222118,10.563901,0.009074,1.258512
7,"Pure Sparkling Water, Lime Sparkling Water","Sparkling Lemon Water, Sparkling Water Grapefruit",0.021026,0.045126,0.010023,0.476701,10.563901,0.009074,1.824722
8,"Pure Sparkling Water, Sparkling Water Grapefruit","Sparkling Lemon Water, Lime Sparkling Water",0.029040,0.040981,0.010023,0.345146,8.422111,0.008833,1.464477
9,"Lime Sparkling Water, Sparkling Water Grapefruit","Sparkling Lemon Water, Pure Sparkling Water",0.055001,0.021421,0.010023,0.182238,8.507480,0.008845,1.196655


In [196]:
## testing the result set
t=test1[test1['antecedants'] =="Organic Strawberries, Organic Baby Spinach"].reset_index()